In [18]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plotly display option
import plotly.io as pio
pd.options.plotting.backend = 'plotly'
pio.renderers.default = "notebook_connected"
pd.options.display.max_rows = 100

### Data preprocessing
- total row 삭제
- object -> int 변경
- Rand Rover 브랜드명 통일

In [2]:
model_df = pd.read_csv("../datas/model_data_yearly_0507.csv")
model_df = model_df.drop(model_df[model_df['BRAND'].str.contains('Total')].index)
model_df['E-D(CC)'] = [int(_) for _ in model_df['E-D(CC)']]
model_df['PRICE'] = [int(_) for _ in model_df['PRICE']]
model_df

,YEAR,MONTH,BRAND,MODEL,E-D(CC),PRICE,SALES,SHARES(%)
0,2016,1,Audi,A1 30 TDI,1598,32700,0,0.00
1,2016,1,Audi,A1 Sportback 30 TDI,1598,33700,0,0.00
2,2016,1,Audi,A3 25 TDI,1598,37500,0,0.00
3,2016,1,Audi,A3 35 TDI,1968,38900,101,0.62
4,2016,1,Audi,A3 35 TFSI,1798,42900,3,0.02
...,...,...,...,...,...,...,...,...
36412,2021,3,Volvo,XC60 T8 AWD,1969,83200,186,0.68
36413,2021,3,Volvo,XC90 B6 AWD,1969,82600,21,0.08
36414,2021,3,Volvo,XC90 D5 AWD,1969,80300,0,0.00
36415,2021,3,Volvo,XC90 T6 AWD,1969,95500,0,0.00


In [3]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34826 entries, 0 to 36416
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   YEAR       34826 non-null  int64  
 1   MONTH      34826 non-null  int64  
 2   BRAND      34826 non-null  object 
 3   MODEL      34826 non-null  object 
 4   E-D(CC)    34826 non-null  int64  
 5   PRICE      34826 non-null  int64  
 6   SALES      34826 non-null  int64  
 7   SHARES(%)  34826 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 2.4+ MB


In [4]:
# 브랜드 개수 26개
br_ls = model_df['BRAND'].unique()
print(len(br_ls))
br_ls

26


array(['Audi', 'BMW', 'Bentley', 'Cadillac', 'Chrysler', 'Citroen',
       'Fiat', 'Ford', 'Honda', 'Infiniti', 'Jaguar', 'Lamborghini',
       'Land Rover', 'Lexus', 'MINI', 'Mercedes-Benz', 'Nissan',
       'Peugeot', 'Porsche', 'Rolls-Royce', 'Toyota', 'Volkswagen',
       'Volvo', 'Maserati', 'Chevrolet', 'Lincoln'], dtype=object)

In [5]:
pv0 = pd.pivot_table(model_df, index=['BRAND','YEAR','MONTH'],
                    aggfunc={'E-D(CC)':'mean', 'PRICE':'mean', 'SALES':sum, 'SHARES(%)':sum}).round(2).reset_index()
pv0

,BRAND,YEAR,MONTH,E-D(CC),PRICE,SALES,SHARES(%)
0,Audi,2016,1,2821.61,94668.78,1900,11.70
1,Audi,2016,2,2778.38,94015.97,984,6.27
2,Audi,2016,3,2778.38,94015.97,2552,10.56
3,Audi,2016,4,2768.35,92449.40,2474,13.86
4,Audi,2016,5,2766.15,93455.50,2336,12.05
...,...,...,...,...,...,...,...
1500,Volvo,2020,11,1969.00,66340.00,1267,4.61
1501,Volvo,2020,12,1969.00,66340.00,1352,4.30
1502,Volvo,2021,1,1969.00,67842.11,1198,5.36
1503,Volvo,2021,2,1969.00,67842.11,1202,5.39


In [6]:
# 브랜드별 연도별 피벗
# MONTH-해당 브랜드의 매출이 발생한 개월 수
# price-해당 기간 동안 팔린 차종의 평균 단가
# SALES-해당 기간 동안 팔린 대수
# SHARES(%)-해당 브랜드 월 평균 점유율

pv1 = pd.pivot_table(pv0, index=['BRAND', 'YEAR'], 
                      aggfunc={'MONTH':'count', 'E-D(CC)':'mean', 'PRICE':'mean', 'SALES':sum, 'SHARES(%)':'mean'},).round().reset_index()
pv1['AVG_SALES'] = round(pv1['SALES']/pv1['MONTH'])
pv1

,BRAND,YEAR,E-D(CC),MONTH,PRICE,SALES,SHARES(%),AVG_SALES
0,Audi,2016,2784.0,12,94586.0,16718,7.0,1393.0
1,Audi,2017,2859.0,12,99543.0,962,0.0,80.0
2,Audi,2018,2876.0,12,100692.0,12450,5.0,1038.0
3,Audi,2019,2344.0,12,77528.0,12072,5.0,1006.0
4,Audi,2020,2299.0,12,79769.0,26069,9.0,2172.0
...,...,...,...,...,...,...,...,...
140,Volvo,2017,2024.0,12,58556.0,6604,3.0,550.0
141,Volvo,2018,2008.0,12,60474.0,8524,3.0,710.0
142,Volvo,2019,1969.0,12,63147.0,10544,4.0,879.0
143,Volvo,2020,1969.0,12,69430.0,14865,6.0,1239.0


### 브랜드별 연도별 월평균 판매량
- 2021년이 3개월 분량만 있으므로 월평균 판매량으로 시각화
- 벤츠가 기간내 전체 1위
- 2위인 BMW: 2019년 잇따른 화재 사고로 판매량 감소
- 3,4위인 아우디, 폭스바겐: 2017년 환경부 인증 문제로 판매 중단 
- (질문)2019년 3위인 셰보레는 왜 그 전엔 데이터가 없지..?
- (질문)E-D(CC) 가 줄어든다는 것은 어떤 의미인가..?

In [7]:
px.line(pv1, x='YEAR', y='AVG_SALES', color='BRAND')

In [8]:
# E-D(CC) 가 줄어든다는 것은 어떤 의미인가..
px.line(pv1.groupby('YEAR').mean(), y='E-D(CC)')

In [9]:
# 브랜드별 pv - 점유율 내림차순
pv2 = pd.pivot_table(pv1,
                     index=['BRAND'], 
                     aggfunc={'E-D(CC)':np.mean,
                              'PRICE':np.mean,
                              'SALES':sum,
                              'SHARES(%)':np.mean}).round(2).reset_index()
pv2 = pv2.sort_values(by=['SHARES(%)'], ascending=False)
pv2

,BRAND,E-D(CC),PRICE,SALES,SHARES(%)
18,Mercedes-Benz,3042.33,118498.83,373090,28.33
1,BMW,2649.50,101408.50,278624,21.50
0,Audi,2619.50,90871.00,75672,6.00
4,Chevrolet,2997.67,41831.00,20887,5.33
24,Volkswagen,2042.00,47900.17,61700,4.83
14,Lexus,3146.67,85715.33,60079,4.33
16,MINI,1829.17,46837.83,52077,4.00
23,Toyota,2395.67,41777.17,56350,3.83
25,Volvo,1998.67,62393.67,49394,3.83
8,Ford,2779.00,56114.50,51599,3.83


### 벤츠를 파보자

#### 우리가 벤츠를 고른 이유

In [10]:
# 우리 조가 벤츠를 고른 이유: 판매량의 28.1% 차지
fig = go.Figure(data=go.Pie(labels=pv2['BRAND'], values=pv2['SALES'], pull=[0.1]))
fig.show()

#### 벤츠는 수입자동차 중에서 어느 정도 위치인가

In [11]:
# 브랜드별 단가
# 벤츠의 단가는 3분위, 즉 상위 25% 수준임
# 단가로만 보면 랜드로버와 재규어와 비슷한 클라스?
fig1 = go.Figure()
fig1.add_trace(
    go.Bar(
        x=pv2.sort_values(by='PRICE', ascending=False)['BRAND'],
        y=pv2.sort_values(by='PRICE', ascending=False)['PRICE']),)
fig1.add_trace(
    go.Box(
        y=pv2['PRICE'],),)

#### 월별 추이

In [12]:
benz0 = pv0[pv0['BRAND']=='Mercedes-Benz']
benz0

,BRAND,YEAR,MONTH,E-D(CC),PRICE,SALES,SHARES(%)
1001,Mercedes-Benz,2016,1,3007.09,99890.24,4298,26.45
1002,Mercedes-Benz,2016,2,3007.09,99923.17,3787,24.20
1003,Mercedes-Benz,2016,3,3007.09,99923.17,5162,21.37
1004,Mercedes-Benz,2016,4,2994.84,99467.47,3558,19.98
1005,Mercedes-Benz,2016,5,3023.42,103187.21,3148,16.21
1006,Mercedes-Benz,2016,6,3057.40,105745.98,4535,19.33
1007,Mercedes-Benz,2016,7,3090.61,109237.50,4184,26.64
1008,Mercedes-Benz,2016,8,3103.09,108960.44,4835,30.38
1009,Mercedes-Benz,2016,9,3116.82,110985.11,5087,30.33
1010,Mercedes-Benz,2016,10,3123.06,111346.39,6400,30.98


In [13]:
# 월별 판매량으로 seasonality를 보려했으나 일정한 규칙이 보이진 않았다
# seasonality 보다는 외부적요인에 의해 구매 여부가 결정되는 듯 하다.
px.line(benz0, x='MONTH', y='SALES', line_group='YEAR', color='YEAR')

In [14]:
# 굳이 추이를 보기위해 월별 평균값 추이를 구해보면
# 6월과 4분기가 판매량이 제일 많다
px.line(benz0.groupby('MONTH').mean(), y='SALES')

#### 연도별 추이
- 매년 판매량 증가 추세
- (질문)2019년을 기점으로 cc가 낮아진 건 어떤 의미일까? 차의 성능이 좋아진 것? 소형차를 많이 사는것..?
- 점유율은 25~31% 사이, 어차피 매년 점유율 1위
- 평균 판매량이 점유율과 비례하진 않는 것으로 보아, 수입차 시장 규모에 영향을 받는 듯

In [15]:
# SALES 2021년은 3개월만 있으므로 월평균 판매량 컬럼 추가
benz1 = pv1[pv1['BRAND']=='Mercedes-Benz']
benz1['AVG_SALES'] = round(benz1['SALES']/benz1['MONTH'])
benz1

,BRAND,YEAR,E-D(CC),MONTH,PRICE,SALES,SHARES(%),AVG_SALES
97,Mercedes-Benz,2016,3072.0,12,106055.0,56343,25.0,4695.0
98,Mercedes-Benz,2017,3205.0,12,116675.0,68861,30.0,5738.0
99,Mercedes-Benz,2018,3239.0,12,123625.0,70798,27.0,5900.0
100,Mercedes-Benz,2019,2993.0,12,117871.0,76032,31.0,6336.0
101,Mercedes-Benz,2020,2916.0,12,122975.0,81834,30.0,6820.0
102,Mercedes-Benz,2021,2829.0,3,123792.0,19222,27.0,6407.0


In [16]:
# 연도별 판매량 추이
# 2021년은 3개월 평균으로 반영되었기에 제외하고 보면 매년 증가 추세
px.bar(benz1, x='YEAR',y='AVG_SALES')

In [17]:
fig2 = make_subplots(rows=3, cols=1)
fig2.add_trace(
    go.Bar(
        x=benz1['YEAR'],y=benz1['AVG_SALES'], name='AVG_SALES'),
        row=1, col=1)
fig2.add_trace(
    go.Scatter(
        x=benz1['YEAR'], y=benz1['E-D(CC)'], name='E-D(CC)'),
        row=2, col=1)
fig2.add_trace(
    go.Scatter(
        x=benz1['YEAR'], y=benz1['SHARES(%)'], name='SHARES(%)'), 
        row=3, col=1)